In [9]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')


In [10]:
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Genre; 
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;


CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);


CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')



In [11]:

fname = 'Library.xml'
if ( len(fname) < 1 ) : 
    fname = 'Library.xml'


In [12]:

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None


In [13]:

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    #added this
    genre = lookup(entry,'Genre')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None or genre is None: 
        continue

    print(name, artist, album, genre, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]
    #Added this
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()
#Added this
sqlstr = 'SELECT Track.title, Artist.name, Album.title, Genre.name FROM Track JOIN Genre JOIN Album JOIN Artist ON Track.genre_id = Genre.ID and Track.album_id = Album.id AND Album.artist_id = Artist.id ORDER BY Artist.name LIMIT 3'



Dict count: 404
Another One Bites The Dust Queen Greatest Hits Rock 55 100 217103
Asche Zu Asche Rammstein Herzeleid Industrial 79 100 231810
Beauty School Dropout Various Grease Soundtrack 48 100 239960
Black Dog Led Zeppelin IV Rock 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] Rock 33 100 87118
Circles Bryan Lee Blues Is Funk 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] Rock 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits Rock 38 100 163631
Electric Funeral Black Sabbath Paranoid Metal 44 100 293015
Fat Bottomed Girls Queen Greatest Hits Rock 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock 84 100 353750
Four Sticks Led Zeppelin IV Rock 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded Soundtrack 54 100 330004
Gelle Bryan Lee Blues Is Blues/R&B 45 60 199836
Going To California Led Zeppelin IV Rock 100 100 215666
Grease Various Grease Soundtrack 42 100 205792
Hand of Doom Black Sabbath Parano

D.T. AC/DC Who Made Who Rock 90 None 173714
Sink the Pink AC/DC Who Made Who Rock 83 None 253466
Ride On AC/DC Who Made Who Rock 75 None 351268
Chase the Ace AC/DC Who Made Who Rock 93 None 181420
Wollt Ihr Das Bett In Flammen Sehen Rammstein Herzeleid Industrial 50 None 317387
Der Meister Rammstein Herzeleid Industrial 46 None 250801
Weisses Fleisch Rammstein Herzeleid Industrial 52 None 215902
Seemann Rammstein Herzeleid Industrial 34 None 288235
Du Riechst So Gut Rammstein Herzeleid Industrial 50 None 289332
Das Alte Leid Rammstein Herzeleid Industrial 46 None 344581
Heirate Mich Rammstein Herzeleid Industrial 39 None 284734
Herzeleid Rammstein Herzeleid Industrial 42 None 223425
Baba O'Riley The Who Who's Next Rock 45 None 298448
Bargain The Who Who's Next Rock 37 None 331676
Love Ain't for Keeping The Who Who's Next Rock 24 None 130560
My Wife The Who Who's Next Rock 17 None 213786
The Song Is Over The Who Who's Next Rock 14 None 375797
Getting In Tune The Who Who's Next Rock 27 N

In [14]:

for row in cur.execute(sqlstr) :
    print(str(row[0]),str(row[1]),str(row[2]),str(row[3]))

cur.close()

For Those About To Rock (We Salute You) AC/DC Who Made Who Rock
Hells Bells AC/DC Who Made Who Rock
Shake Your Foundations AC/DC Who Made Who Rock


In [15]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor()


In [16]:

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')



In [17]:
fname = 'roster_data.json'
if len(fname) < 1:
    fname = 'roster_data.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0]
    title = entry[1]
    role = entry[2]

    print(name, title)

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', (name, ))
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', (title, ))
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        (user_id, course_id, role))

    conn.commit()

Aphra si110
Ray si110
Carys si110
Keisha si110
Shantelle si110
Kaylin si110
Bob si110
Zerah si110
Charly si110
Keigan si110
Tymon si110
Caidie si110
Susannah si110
Maggie si110
Halina si110
Ryleigh si110
Fergus si110
Jazz si110
Zenah si110
Leilani si110
Charly si110
Khadijah si110
Kennedy si110
Mercy si110
Dorothy si110
Simah si110
Carly si110
Monique si110
Cael si110
Jadyn si110
Leena si110
Taiwo si110
Olivier si110
Dakotah si110
Eoghain si106
Filip si106
Estelle si106
Jacob si106
Kaan si106
Kaylem si106
Kahlea si106
Alessandro si106
Codey si106
Lacey si106
Jaydyn si106
Amro si106
Renas si106
Demetrius si106
Erik si106
Brehme si106
Mena si106
Eidhan si106
Sayeda si106
Dillon si106
Coby si206
Shikha si206
Musa si206
Betty si206
Matas si206
Raina si206
Aaliyah si206
Summer si206
Asha si206
Vivian si206
Marwa si206
Aamina si206
Ruslan si206
Kristal si206
Sorley si206
Taniesha si206
Chase si206
Payton si206
Michee si206
Cacie si206
Sally si206
Seamas si206
Reily si206
Kennedy si206
Carrag

In [18]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")
